# Projet 4

In [241]:
import requests
from bs4 import BeautifulSoup
import re
import json
import time
from multiprocessing import Pool

Ecrire une fonction `get_prices_from_url()` qui extrait des informations à partir des 2 pages ci-dessous.

Exemple `URL_PAGE2` doit retourner :

<pre>{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
  'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
  'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}
</pre>

In [242]:
URL_PAGE2 = "https://kim.fspot.org/cours/page2.html"
URL_PAGE3 = "https://kim.fspot.org/cours/page3.html"

def get_prices_from_url(url):
    prices = {}
    
    request = requests.get(url)
    soup = BeautifulSoup(requests.get(url).content,'html')
    
    tag_price = soup.find_all(class_="pricing-table-header")
    tag_li = soup.find_all(class_="pricing-table-list")
    
    categories_li = []
    price_li = []
    storage_li = []
    database_li = []
    dict_li = []
    
    for i in range(len(tag_price)):
        categories_li.append(tag_price[i].h2.text)
        price_li.append(tag_price[i].find(string=re.compile("\$\d+")).strip())
        storage_li.append(re.match("\d+\wB", tag_li[i].find(string=re.compile("storage")))[0])
        database_li.append(int(re.match("\d+", tag_li[i].find(string=re.compile("database")))[0]))
    for n, j, s in zip(price_li, storage_li, database_li):
        dict_el = {'price': n, 'storage': j, 'databases': s}
        dict_li.append(dict_el)
    prices = {categories_li[k]: dict_li[k] for k in range(len(dict_li))}  
     
    
    return prices

In [243]:
#test get_prices_from_url
get_prices_from_url(URL_PAGE2)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25}}

In [244]:
get_prices_from_url(URL_PAGE3)

{'Personal': {'price': '$5', 'storage': '1GB', 'databases': 1},
 'Small Business': {'price': '$25', 'storage': '10GB', 'databases': 5},
 'Enterprise': {'price': '$45', 'storage': '100GB', 'databases': 25},
 'Privilege': {'price': '$99', 'storage': '1TB', 'databases': 100}}

Ecrire une fonction qui extrait des informations sur une bière de beowulf.

Exemple d'URL: https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33 

La fonction doit retourner :
<pre>
{'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33}
</pre>

In [245]:
def extract_beer_infos(url):
    infos = {
        'name': None,
        'note': None,
        'price': None,
        'volume': None,
    }
        
    soup = BeautifulSoup(requests.get(url).content, 'lxml')
    name = soup.find('h1').text.strip()
    price = float(soup.find('span', attrs = {"class":"price"}).text[0:-1].strip().replace(',','.'))
    note = soup.find('div', attrs = {"class":"stars"})['data-percent']
    volume = soup.find('dd', attrs = {"class":"small-6 medium-9 columns js-beer-volume"})['data-volume']
    for s in re.findall(r'-?\d+\.?\d*', volume):
        volume = int(s) 
    
    infos = {'name': name, 'note': note, 'price': price, 'volume': volume}
    
    return infos

In [246]:
URL_MELUSINE = "https://www.beerwulf.com/fr-fr/p/bieres/melusine-bio.33"

extract_beer_infos(URL_MELUSINE)

{'name': 'Mélusine Bio', 'note': '70', 'price': 38.99, 'volume': 33}

Cette URL retourne un JSON avec une liste de bières :

In [247]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

Ecrire une fonction qui prend l'argument cet URL retourne les informations sur une liste de bière via l'API de beowulf.

Cette fonction doit retourner la liste des informations obtenues par la fonction `extract_beer_infos()` définie ci-dessus.

Chercher comment optimiser cette fonction en utilisant multiprocessing.Pool pour paralléliser les accès web.

Exemple de retour :

<pre>[{'name': 'Gallia East IPA', 'note': 80, 'price': 42.99, 'volume': 33},
    {'name': 'La Lager Sans Gluten de Vézelay', 'note': 60, 'price': 38.99, 'volume': 25},
    {'name': 'Brasserie De Sutter Brin de Folie', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'La Cristal IPA du Mont Blanc', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Mélusine Bio', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'La Parisienne Le Titi Parisien', 'note': 70, 'price': 38.99, 'volume': 33},
    {'name': 'Gallia Session IPA', 'note': 70, 'price': 42.99, 'volume': 33},
    {'name': 'Ninkasi Brut IPA', 'note': 70, 'price': 44.99, 'volume': 33},
    {'name': 'Pietra', 'note': 60, 'price': 38.99, 'volume': 33},
    {'name': 'Desperados', 'note': 60, 'price': 35.99, 'volume': 33},
    {'name': 'Gallia West IPA', 'note': 70, 'price': 42.99, 'volume': 33}]
</pre>

In [248]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    beer_pages = []
    
    # Sequential version (slow):
    beers = []

    # Parallel version (faster):
    # beers = []
    
    return beers

In [249]:
def extract_beer_list_infos(url):
    # Collecter les pages de bières à partir du JSON
    pool = multiprocessing.Pool()
    
    req = requests.get(url)
    db = json.loads(req.content)['items']
    
    beer_urls = []
    beers = []
   
    #Extract the URLs of the list of beers
    for i in range(len(db)):
        beer_urls.append("https://www.beerwulf.com" + db[i]['contentReference'])
        
    # Sequential version (slow):
    start = time.time()
    for url in beer_urls:
        beers.append(extract_beer_infos(url))
    end = time.time()
    
    print(f"Time of extracting data using sequential version is {end - start}")
    
    # Parallel version (faster):
#     start = time.time()
#     with Pool() as pool:
#         beers = pool.map(extract_beer_infos, beer_urls)
#     end = time.time()
#     print(f"Time of extracting data using parallel version is {end - start}")

    return beers

In [250]:
URL_BEERLIST_FRANCE = "https://www.beerwulf.com/fr-FR/api/search/searchProducts?country=France&container=Bouteille"

extract_beer_list_infos(URL_BEERLIST_FRANCE)

Time of extracting data using sequential version is 2.153733015060425


[{'name': 'Gallia East IPA', 'note': '80', 'price': 42.99, 'volume': 33},
 {'name': 'La Lager Sans Gluten de Vézelay',
  'note': '60',
  'price': 38.99,
  'volume': 25},
 {'name': 'Brasserie De Sutter Brin de Folie',
  'note': '70',
  'price': 44.99,
  'volume': 33},
 {'name': 'La Cristal IPA du Mont Blanc',
  'note': '70',
  'price': 44.99,
  'volume': 33},
 {'name': 'Mélusine Bio', 'note': '70', 'price': 38.99, 'volume': 33},
 {'name': 'La Parisienne Le Titi Parisien',
  'note': '70',
  'price': 38.99,
  'volume': 33},
 {'name': 'Gallia Session IPA', 'note': '70', 'price': 42.99, 'volume': 33},
 {'name': 'Ninkasi Brut IPA', 'note': '70', 'price': 44.99, 'volume': 33},
 {'name': 'Pietra', 'note': '60', 'price': 38.99, 'volume': 33},
 {'name': 'Desperados', 'note': '60', 'price': 35.99, 'volume': 33},
 {'name': 'Gallia West IPA', 'note': '70', 'price': 42.99, 'volume': 33}]

In [251]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_01_get_prices_from_url_page2(self):
        prices = get_prices_from_url(URL_PAGE2)
        # We should have found 3 products:
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 3)
        self.assertIn('Personal', prices)
        self.assertIn('Small Business', prices)
        self.assertIn('Enterprise', prices)
        
        personal = prices['Personal']
        self.assertIn('price', personal)
        self.assertIn('storage', personal)
        self.assertIn('databases', personal)
        self.assertEqual(personal['price'], '$5')
        self.assertEqual(personal['storage'], '1GB')
        self.assertEqual(personal['databases'], 1)
        
    def test_02_get_prices_from_url_page3(self):
        prices = get_prices_from_url(URL_PAGE3)
        self.assertIsInstance(prices, dict)
        self.assertEqual(len(prices), 4)
        self.assertEqual(
            prices['Privilege'],
            {'databases': 100, 'price': '$99', 'storage': '1TB'}
        )
    
    def test_03_extract_beer_list_infos(self):
        infos = extract_beer_list_infos(URL_BEERLIST_FRANCE)
        # We should have 11 French beers:
        self.assertIsInstance(infos, list)
        self.assertEqual(len(infos), 11)
        # All of them are 25cl or 33cl:
        for beer in infos:
            self.assertIn(beer['volume'], [25, 33])

            
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [252]:
if __name__ == '__main__':
    run_tests()

test_01_get_prices_from_url_page2 (__main__.Lesson4Tests) ... ok
test_02_get_prices_from_url_page3 (__main__.Lesson4Tests) ... ok
test_03_extract_beer_list_infos (__main__.Lesson4Tests) ... 

Time of extracting data using sequential version is 2.0535240173339844


ok

----------------------------------------------------------------------
Ran 3 tests in 2.713s

OK
